In [ ]:
# Detectron2 및 PyBullet 설치
!pip install torch torchvision
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!apt-get install -y bullet
!pip install pybullet
!pip install opencv-python-headless matplotlib numpy


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-dx_7updo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-dx_7updo
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.1 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [ ]:
# 라이브러리 불러오기
import cv2
import numpy as np
import torch
import pybullet as p
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Detectron2를 통한 이미지 내 당구공 탐지 함수
def detect_balls(image_path):
    # 모델 설정
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # 정확도 기준
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)

    # 이미지 불러오기 및 탐지
    image = cv2.imread(image_path)
    outputs = predictor(image)
    return outputs["instances"].pred_boxes.tensor.cpu().numpy(), outputs["instances"].pred_classes.cpu().numpy()

# 당구대 위 공의 궤적 계산
def simulate_shot(cue_ball, target_ball_1, target_ball_2, cushion_count):
    # PyBullet 시뮬레이션 초기화
    p.connect(p.DIRECT)
    p.setGravity(0, 0, -10)

    # 당구 테이블 크기 설정 (단위: 미터)
    table_width = 2.84
    table_height = 1.42
    cue_ball_pos = np.array(cue_ball)
    target_1_pos = np.array(target_ball_1)
    target_2_pos = np.array(target_ball_2)

    # PyBullet 환경 설정
    # 공 생성 및 물리 설정
    cue_ball_id = p.createCollisionShape(p.GEOM_SPHERE, radius=0.028)
    p.createMultiBody(0.17, cue_ball_id, basePosition=cue_ball_pos.tolist())

    # 시뮬레이션 루프
    for i in range(240):
        p.stepSimulation()

        # 조건 충족 시(쿠션, 맞힘 여부) 루프 종료
        if is_collision(cue_ball_pos, target_1_pos):
            print("1적구를 맞혔습니다!")
            break

    p.disconnect()

# 공 충돌 여부 확인
def is_collision(pos1, pos2, threshold=0.03):
    return np.linalg.norm(np.array(pos1) - np.array(pos2)) < threshold

# 시각화 함수
def visualize_shot(image_path, trajectory):
    image = cv2.imread(image_path)
    for point in trajectory:
        cv2.circle(image, tuple(map(int, point)), 5, (0, 255, 0), -1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

# 전체 흐름
def main():
    image_path = "/content/drive/MyDrive/AIFFEL/Aiffelthon/ModelTest/onlyball1.png"
    boxes, classes = detect_balls(image_path)

    # 공의 탐지 여부 확인
    print(f"Detected boxes: {boxes}, Classes: {classes}, Number of balls detected: {len(boxes)}")

    if len(boxes) < 3:
        print("공이 충분히 탐지되지 않았습니다. 이미지 품질을 확인하거나 탐지 임계값을 낮춰보세요.")
        return

    # 좌표 매핑
    cue_ball, target_ball_1, target_ball_2 = boxes[0], boxes[1], boxes[2]

    simulate_shot(cue_ball, target_ball_1, target_ball_2, cushion_count=2)
    visualize_shot(image_path, [(cue_ball[0], cue_ball[1])])

main()


ModuleNotFoundError: No module named 'pybullet'

In [ ]:
def example_function():
    # ... 일부 로직 ...
    if condition:
        return  # 함수 내부에서 사용
